In [13]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

### Initializing the Model

In [40]:
m = pyo.ConcreteModel()

### Set Parameteres (What is given?)

- Name of the projects
- Number of days to do it
- Time taken to do each project
- Profit for each of the projects
- Max hours that can be worked on each day

In [41]:
m.setJ = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E',
                             'F'])  #different projects to choose from
m.setD = pyo.Set(initialize=[1, 2, 3])  #number of working days
m.D = {
    'A': 2,
    'B': 3,
    'C': 5,
    'D': 2,
    'E': 6,
    'F': 4
}  #number of duration for each project
m.P = {
    'A': 200,
    'B': 500,
    'C': 300,
    'D': 100,
    'E': 1000,
    'F': 300
}  #amount of profit from each project
m.maxHours = 6 #number of hours working everyday
#m.maxJobDay = 1  #number of projects per day 

### Variables (What do we need to find)?

- Ans: Which projects we will do on which day.
    - J can take [A,B,C,D,E,F]
    - D can take [1,2,3]
    - x[J,D] = 0 or 1

In [42]:
m.x = pyo.Var(m.setJ, m.setD, within=pyo.Binary)

#### Example: possible_combination 

In [43]:
jobs = ['A', 'B', 'C', 'D', 'E','F']
days = [1,2,3]
possible_combination = [ [d,j] for d in days for j in jobs ]
possible_combination

[[1, 'A'],
 [1, 'B'],
 [1, 'C'],
 [1, 'D'],
 [1, 'E'],
 [1, 'F'],
 [2, 'A'],
 [2, 'B'],
 [2, 'C'],
 [2, 'D'],
 [2, 'E'],
 [2, 'F'],
 [3, 'A'],
 [3, 'B'],
 [3, 'C'],
 [3, 'D'],
 [3, 'E'],
 [3, 'F']]

### Objective Function (How do we find the best possible outcome)?

In [44]:
#m.x[j,d]*m.P[j] says if a project A is done on day 1 then m.x[j,d]==1 and m.P['A']==200
m.obj = pyo.Objective(expr=sum(
    [m.x[j, d] * m.P[j] for j in m.setJ for d in m.setD]),
                      sense=pyo.maximize)

### constraints

In [45]:
m.C1 = pyo.ConstraintList()
m.C2 = pyo.ConstraintList()
m.C3 = pyo.ConstraintList()
for d in m.setD:
    m.C1.add(sum([m.x[j,d]*m.D[j] for j in m.setJ]) <= m.maxHours) #for each day total hours sould be less than 6
    
for j in m.setJ:
    m.C2.add(sum([m.x[j,d] for d in m.setD]) <= 1) #one job can be done only on one of the days
    
#for d in m.setD:
    #m.C3.add(sum(m.x[j,d] for j in m.setJ) ==1) #constrain condition to have only one job per day

### Solveing using Solver

In [46]:
opt = SolverFactory('glpk')
m.results = opt.solve(m)

#print
m.pprint()

6 Set Declarations
    C1_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    C2_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}
    setD : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    setJ : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'A', 'B', 'C', 'D', 'E', 'F'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain    : Size : Members
        None :     2 : setJ*setD :   18 : {('A', 1), ('A', 2), ('A', 3), ('B', 1), ('B', 2), ('B', 3), ('C', 1), ('C', 

In [47]:
print('\n\n')
print('Profit Total:', pyo.value(m.obj))
for d in m.setD:
    for j in m.setJ:
        if pyo.value(m.x[j, d]) > 0.9:
            print('Job %s in day %d (duration %i, profit %i)' %
                  (j, d, m.D[j], m.P[j]))




Profit Total: 2100.0
Job E in day 1 (duration 6, profit 1000)
Job B in day 2 (duration 3, profit 500)
Job D in day 2 (duration 2, profit 100)
Job A in day 3 (duration 2, profit 200)
Job F in day 3 (duration 4, profit 300)
